<a href="https://colab.research.google.com/github/vishallmaurya/sentiment_analyzer_backend/blob/main/sentiment_analysis3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import re
import nltk

from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sentence_transformers import SentenceTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import svm

In [3]:
data = pd.read_csv('/content/sentiment_data_label.csv')
data1 = pd.read_csv('/content/sentiment_data_2_label.csv')
data2 = pd.read_csv('/content/sentiment_data_3csv.csv')

In [4]:
data = pd.concat([data, data1, data2])

In [5]:
data.head()

,Unnamed: 0,Date,Tweet,class
0,0,2022-12-31 23:59:51+00:00,So reuniting with an Ex was special and Im sur...,0
1,1,2022-12-31 23:59:11+00:00,#Happy #New #Year!!! Hope you all are having a...,1
2,2,2022-12-31 23:53:18+00:00,@TeamSMATh #happy new year 2023 SMATH,1
3,3,2022-12-31 23:52:30+00:00,A fresh start this new year will bring.\nAn op...,1
4,4,2022-12-31 23:45:50+00:00,"Sometimes it feels life is unfair, sometimes i...",1


In [6]:
data.drop(['Unnamed: 0', 'Date'], inplace=True, axis=1)

In [7]:
nltk.download('stopwords')

all_stopwords = stopwords.words('english')
ps = PorterStemmer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [8]:
def cleaning_tweets(tweet):
  tweet = re.sub('[^a-zA-Z]', ' ', tweet)
  tweet = tweet.lower()
  tweet = tweet.split()
  tweet = [ps.stem(word) for word in tweet if not word in set(all_stopwords)]
  tweet = ' '.join(tweet)
  return tweet

In [9]:
d = []
for i in range(data.shape[0]):
  d.append(cleaning_tweets(str(data.iloc[i]["Tweet"])))

In [10]:
model = SentenceTransformer("all-MiniLM-L6-v2")

tweet_embeddings = model.encode(d, convert_to_numpy=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [15]:
from imblearn.over_sampling import SMOTE

In [16]:
SMOTE = SMOTE()

X_SMOTE, Y_SMOTE = SMOTE.fit_resample(tweet_embeddings, data["class"])

In [17]:
X_SMOTE.shape

(1224, 384)

In [18]:
X_train, X_test, y_train, y_test = train_test_split(tweet_embeddings, data["class"], test_size=0.2, random_state=42)
X_train1, X_test1, y_train1, y_test1 = train_test_split(X_SMOTE, Y_SMOTE, test_size=0.2, random_state=42)


In [19]:
from sklearn.model_selection import cross_val_score

In [20]:
# Train an SVM model on the embeddings
svm_model = svm.SVC(kernel="rbf", C=10, probability=True)
svm_model.fit(X_train, y_train)

# Evaluate the model
y_pred = svm_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy:.4f}")


Model Accuracy: 0.8150


In [33]:
# Train an SVM model on the embeddings
svm_model1 = svm.SVC(kernel="rbf", C=10, probability=True)
svm_model1.fit(X_train1, y_train1)

# Evaluate the model
y_pred1 = svm_model1.predict(X_test1)
accuracy = accuracy_score(y_test1, y_pred1)
print(f"Model Accuracy: {accuracy:.4f}")


Model Accuracy: 0.8857


In [13]:
svm_model2 = svm.SVC(kernel="rbf", C=10, probability=True)
score3 = cross_val_score(svm_model2, tweet_embeddings, data["class"], cv=50)

acc_svm = score3.mean()*100

print("Accuracy on the test data ", acc_svm, "%")

Accuracy on the test data  78.39215686274508 %


In [21]:
svm_model2 = svm.SVC(kernel="rbf", C=10, probability=True)
score3 = cross_val_score(svm_model2, X_SMOTE, Y_SMOTE, cv=50)

acc_svm = score3.mean()*100

print("Accuracy on the test data ", acc_svm, "%")

Accuracy on the test data  89.60333333333332 %


In [42]:
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier

In [37]:
# maxi = 0
# esti = 1
# for i in range(1, 210):
#   rf_model = RandomForestClassifier(n_estimators=i, random_state=42)
#   rf_model.fit(X_train1, y_train1)

#   # Evaluate accuracy
#   y_pred1 = rf_model.predict(X_test1)
#   val = accuracy_score(y_test1, y_pred1)
#   print(f"Random Forest Accuracy: {val:.4f}")
#   if val > maxi:
#     maxi = val
#     esti = i

# print("Maxi:  ", maxi)
# print("esit: " , esti)

In [51]:
# maxi = 0
# esti = 1
# for i in range(150, 200):
#   rf_model = ExtraTreesClassifier(n_estimators=i, warm_start=True, random_state=42)
#   rf_model.fit(X_train1, y_train1)

#   # Evaluate accuracy
#   y_pred1 = rf_model.predict(X_test1)
#   val = accuracy_score(y_test1, y_pred1)
#   # print(f"Random Forest Accuracy: {val:.4f}")
#   print(i)
#   if val > maxi:
#     maxi = val
#     esti = i

# print("Maxi:  ", maxi)
# print("esit: " , esti)

In [49]:
rf_model = ExtraTreesClassifier(n_estimators=161,warm_start=True, random_state=42)
rf_model.fit(X_train1, y_train1)

# Evaluate accuracy
y_pred1 = rf_model.predict(X_test1)
print(f"Extra Forest Accuracy: {accuracy_score(y_test1, y_pred1):.4f}")

Extra Forest Accuracy: 0.9143


In [48]:
rf_model = RandomForestClassifier(n_estimators=125, random_state=42)
rf_model.fit(X_train1, y_train1)

# Evaluate accuracy
y_pred1 = rf_model.predict(X_test1)
print(f"Random Forest Accuracy: {accuracy_score(y_test1, y_pred1):.4f}")

Random Forest Accuracy: 0.9184


In [39]:
rf_model = RandomForestClassifier(n_estimators=125, random_state=42)
rf_model.fit(X_train, y_train)

# Evaluate accuracy
y_pred = rf_model.predict(X_test)
print(f"Random Forest Accuracy: {accuracy_score(y_test, y_pred):.4f}")

Random Forest Accuracy: 0.8035


In [40]:
rf_model = RandomForestClassifier(n_estimators=125, random_state=42)
score3 = cross_val_score(rf_model, X_SMOTE, Y_SMOTE, cv=50)

acc_svm = score3.mean()*100

print("Accuracy on the test data ", acc_svm, "%")

Accuracy on the test data  88.39666666666668 %


In [41]:
rf_model = RandomForestClassifier(n_estimators=125, random_state=42)
score3 = cross_val_score(rf_model, tweet_embeddings, data["class"], cv=50)

acc_svm = score3.mean()*100

print("Accuracy on the test data ", acc_svm, "%")

Accuracy on the test data  76.93464052287581 %
